In [1]:
from atgreen import *

In [2]:
PATH=""

In [ ]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']


for ind,city in enumerate(cities_list):
    print(ind)
    print(city)

    dfs_list=[]
    for key,value in osm_greencombinations.items():
        df=query4osm2grid(city, 'category', value, db_params, 0, 0)
        # Round values
        for VAR in ['green_size', 'size_intersection']:
            df[VAR]=np.round(df[VAR], 2)
        #Rename variables
        df.rename(columns={"green_size":f"{key}_gs", "size_intersection":f"{key}_si"}, inplace=True)
        df['xy_count']=df.groupby(['x', 'y']).cumcount()
        #Generate fake additional variable for merge - it is used to ensure that elements are not duplicated during the merge
        dfs_list.append(df)

    df=dfs_list[0].copy()
    for i in dfs_list[1:]:
        df=pd.merge(df, i, on=['x','y', 'xy_count'], how='outer')
    df=df.fillna(0)
    df.drop(columns=['xy_count'], inplace=True)
    df['city']=city
    df2psql(df,"osm.osm2grid", db_params, if_exists='append', index=False, index_label=['x','y', 'city'])